# Task 4: Sequential Pattern mining

## Model the customer as a sequence of baskets  

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datetime import datetime
import zipfile

zf = zipfile.ZipFile("/content/drive/My Drive/DataMing/customer_supermarket.csv.zip") 
df = pd.read_csv(zf.open('customer_supermarket.csv'), sep='\t', index_col=0, decimal=',', parse_dates=['BasketDate'])
# Delete all records with 'ProdID' which are not start with a number
df = df[df['ProdID'].str.startswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9'))]
# Delete all return records
df = df[df['BasketID'].str.startswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9'))]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460875 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   BasketID         460875 non-null  object        
 1   BasketDate       460875 non-null  datetime64[ns]
 2   Sale             460875 non-null  float64       
 3   CustomerID       396370 non-null  object        
 4   CustomerCountry  460875 non-null  object        
 5   ProdID           460875 non-null  object        
 6   ProdDescr        460128 non-null  object        
 7   Qta              460875 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 31.6+ MB


In [ ]:
# Take the first five bits of 'ProdID'.
df['ProdID'] = df['ProdID'].apply(lambda x:x[:5])
# Number of total 'ProdID' categories.
len(df['ProdID'].unique()),len(df['ProdDescr'].unique())

(3311, 4058)

In [ ]:
# Transform 'BasketDate' into 'Date'.
date_split=df['BasketDate'].astype('str').str.split(' ')
df['Date']=date_split.str[0]
# Parse the variable 'Date' from type 'str' to type 'datetime64[ns]'.
df.loc[:,'Date']=pd.to_datetime(df.loc[:,'Date'], format='%Y-%m-%d', errors='coerce')
Min_date = df.Date.min()
df['Timestamp'] = pd.Series(df['Date'] - Min_date).dt.days
df.tail()

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta,Date,Timestamp
541904,581587,2011-09-12 12:50:00,0.85,12680.0,France,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-09-12,608
541905,581587,2011-09-12 12:50:00,2.10,12680.0,France,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-09-12,608
541906,581587,2011-09-12 12:50:00,4.15,12680.0,France,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-09-12,608
541907,581587,2011-09-12 12:50:00,4.15,12680.0,France,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-09-12,608
541908,581587,2011-09-12 12:50:00,4.95,12680.0,France,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-09-12,608


In [ ]:
# Generate itemsets.
transactions = df.groupby(['CustomerID','Timestamp','BasketID'])['ProdID'].apply(list).reset_index(name='ProdIDs')
transactions.head()

,CustomerID,Timestamp,BasketID,ProdIDs
0,12346.0,371,541431,[23166]
1,12347.0,181,537626,"[85116, 22375, 71477, 22492, 22771, 22772, 227..."
2,12347.0,379,542237,"[84625, 84625, 85116, 20719, 22375, 22376, 209..."
3,12347.0,392,562032,"[23308, 23297, 22375, 22374, 22376, 22371, 223..."
4,12347.0,538,549222,"[22376, 22374, 22371, 22375, 20665, 23076, 217..."


In [ ]:
# Generate sequences which sequence lenght large than one
sequences = transactions.groupby(['CustomerID'])['ProdIDs'].apply(list).reset_index()
sequences = sequences[sequences.ProdIDs.str.len()>1]
sequences = sequences.ProdIDs.to_list()
len(sequences)

2829

In [ ]:
# Write the sequences into input file in SPMF format. 
with open(r"/content/drive/My Drive/DataMing/SequencesNew5.txt", "w") as text_file:
  for sequence in sequences:
    for element in sequence:
      for i,item in enumerate(element):
        text_file.write(str(item) + " ")
      text_file.write("-1 ")
    text_file.write("-2\n")

**The input file format :**

It is a text file where each line represents a sequence from a sequence database. Each item from a sequence is a positive integer and items from the same itemset within a sequence are separated by single space. Note that it is assumed that items within a same itemset are sorted according to a total order and that no item can appear twice in the same itemset. The value "-1" indicates the end of an itemset. The value "-2" indicates the end of a sequence (it appears at the end of each line).

**Optional feature: giving names to items**

Some users have requested the feature of given names to items instead of using numbers. This feature is offered in the user interface of SPMF and in the command line of SPMF. To use this feature, your file must include @CONVERTED_FROM_TEXT as first line and then several lines to define the names of items in your file. 

In [ ]:
# # Write the sequences into input file in SPMF format.
# # Giving ProdDescr to corresponding ProdID from text, filter out outliers such as 'nan', 'Damaged', 're-adjustment', 'Unsaleable, destroyed.', 'mailout'. 
# with open(r"/content/drive/My Drive/DataMing/SequencesNew5C.txt", "w") as text_file:
#   text_file.write('@CONVERTED_FROM_TEXT\n')
#   for PID in df.ProdID.unique():
#     temp = [x for x in df[df['ProdID'].str.startswith((PID))].ProdDescr.unique() if str(x) != 'nan'and str(x) != 'mailout' and str(x) != 'Damaged' and str(x) != 're-adjustment' and str(x) != 'Unsaleable, destroyed.']
#     if temp != []:
#       text_file.write('@ITEM=' + PID + '=' + temp[0].replace(" ","_") + '\n')
#   for sequence in sequences:
#     for element in sequence:
#       for i,item in enumerate(element):
#         text_file.write(str(item) + " ")
#       text_file.write("-1 ")
#     text_file.write("-2\n")

## Apply the sequential pattern mining algorithm 

In [ ]:
# Install spmf tool
# Remember to download 'spmf.jar' from (http://www.philippe-fournier-viger.com/spmf/download-spmfjar.php) and put it into 'spmf_bin_location_dir'.
!pip3 install spmf
from spmf import Spmf

**PrefixSpan** discovers all frequent sequential patterns occurring in a sequence database (subsequences that occurs in more than minsup sequences of the database.
**PrefixSpan** is one of the fastest sequential pattern mining algorithm. 

**Parameter:**
* minimum support (**minsup**): the minimum number of sequences that should contain a sequential patterns (a positive integer >=0)

In [ ]:
# PrefixSpan algorithm
spmf = Spmf("PrefixSpan", spmf_bin_location_dir = r"/content/drive/My Drive/DataMing/", input_filename= r"/content/drive/My Drive/DataMing/SequencesNew5.txt",
            output_filename= r"/content/drive/My Drive/DataMing/output.txt", arguments=[0.05])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("/content/drive/My Drive/DataMing/outputPrefixSpan_5C.txt")

>/content/drive/My Drive/DataMing/spmf.jar
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 2579 ms
 Frequent sequences count : 771
 Max memory (mb) : 157.15345764160156
 minsup = 142 sequences.
 Pattern count : 771


            pattern  sup
0           [82483]  223
1           [82482]  365
2    [82482, 85123]  147
3    [82482, 82482]  179
4    [82482, 82494]  169
..              ...  ...
766         [48184]  202
767         [48187]  274
768         [48188]  147
769         [48194]  254
770         [16161]  247

[771 rows x 2 columns]


**The output file format :**

It is a text file. Each line is a frequent sequential pattern. Each item from a sequential pattern is a positive integer and items from the same itemset within a sequence are separated by single spaces. The value "-1" indicates the end of an itemset. On each line, the sequential pattern is first indicated. Then, the keyword "#SUP:" appears followed by an integer indicating the support of the pattern as a number of sequences.

 ## Discuss the resulting patterns 

## Optional (2 points): Extend the algorithm and analysis considering one or more time constraints.


### Model the customer as a sequence of baskets

In [ ]:
from datetime import datetime
import zipfile

zf = zipfile.ZipFile("/content/drive/My Drive/DataMing/customer_supermarket.csv.zip") 
df = pd.read_csv(zf.open('customer_supermarket.csv'), sep='\t', index_col=0, decimal=',', parse_dates=['BasketDate'])
# Delete all records with 'ProdID' which are not start with a number
df = df[df['ProdID'].str.startswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9'))]
# Delete all return records
df = df[df['BasketID'].str.startswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9'))]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460875 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   BasketID         460875 non-null  object        
 1   BasketDate       460875 non-null  datetime64[ns]
 2   Sale             460875 non-null  float64       
 3   CustomerID       396370 non-null  object        
 4   CustomerCountry  460875 non-null  object        
 5   ProdID           460875 non-null  object        
 6   ProdDescr        460128 non-null  object        
 7   Qta              460875 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 31.6+ MB


In [ ]:
# Take the first five bits of 'ProdID'.
df['ProdID'] = df['ProdID'].apply(lambda x:x[:5])
# Number of total 'ProdID' categories.
len(df['ProdID'].unique())

3311

In [ ]:
# Transform 'BasketDate' into 'Date'.
date_split=df['BasketDate'].astype('str').str.split(' ')
df['Date']=date_split.str[0]
# Parse the variable 'Date' from type 'str' to type 'datetime64[ns]'.
df.loc[:,'Date']=pd.to_datetime(df.loc[:,'Date'], format='%Y-%m-%d', errors='coerce')
Min_date = df.Date.min()
df['Timestamp'] = pd.Series(df['Date'] - Min_date).dt.days
df.tail()

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta,Date,Timestamp
541904,581587,2011-09-12 12:50:00,0.85,12680.0,France,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-09-12,608
541905,581587,2011-09-12 12:50:00,2.10,12680.0,France,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-09-12,608
541906,581587,2011-09-12 12:50:00,4.15,12680.0,France,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-09-12,608
541907,581587,2011-09-12 12:50:00,4.15,12680.0,France,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-09-12,608
541908,581587,2011-09-12 12:50:00,4.95,12680.0,France,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-09-12,608


In [ ]:
# Generate itemsets.
transactions = df.groupby(['CustomerID','Timestamp','BasketID'])['ProdID'].apply(list).reset_index(name='ProdIDs')
transactions.head()

,CustomerID,Timestamp,BasketID,ProdIDs
0,12346.0,371,541431,[23166]
1,12347.0,181,537626,"[85116, 22375, 71477, 22492, 22771, 22772, 227..."
2,12347.0,379,542237,"[84625, 84625, 85116, 20719, 22375, 22376, 209..."
3,12347.0,392,562032,"[23308, 23297, 22375, 22374, 22376, 22371, 223..."
4,12347.0,538,549222,"[22376, 22374, 22371, 22375, 20665, 23076, 217..."


In [ ]:
# Insert time information for each Basket.
for i in range(len(transactions.ProdIDs)):
 transactions.ProdIDs[i].insert(0, transactions.Timestamp[i].astype(str))
transactions.head()

,CustomerID,Timestamp,BasketID,ProdIDs
0,12346.0,371,541431,"[371, 23166]"
1,12347.0,181,537626,"[181, 85116, 22375, 71477, 22492, 22771, 22772..."
2,12347.0,379,542237,"[379, 84625, 84625, 85116, 20719, 22375, 22376..."
3,12347.0,392,562032,"[392, 23308, 23297, 22375, 22374, 22376, 22371..."
4,12347.0,538,549222,"[538, 22376, 22374, 22371, 22375, 20665, 23076..."


In [ ]:
# Generate sequences which sequence lenght large than one
sequences = transactions.groupby(['CustomerID'])['ProdIDs'].apply(list).reset_index()
sequences = sequences[sequences.ProdIDs.str.len()>1]
sequences = sequences.ProdIDs.to_list()
len(sequences)

2829

In [ ]:
# # Write the sequences into input file in SPMF format.
# # Giving ProdDescr to corresponding ProdID from text, filter out outliers such as 'nan', 'Damaged', 're-adjustment', 'Unsaleable, destroyed.', 'mailout'. 
# with open(r"/content/drive/My Drive/DataMing/SequencesWithTimeNew5C.txt", "w") as text_file:
#   text_file.write('@CONVERTED_FROM_TEXT\n')
#   for PID in df.ProdID.unique():
#     temp = [x for x in df[df['ProdID'].str.startswith((PID))].ProdDescr.unique() if str(x) != 'nan' and str(x) != 'mailout' and str(x) != 'Damaged' and str(x) != 're-adjustment' and str(x) != 'Unsaleable, destroyed.']
#     if temp != []:
#       text_file.write('@ITEM=' + PID + '=' + temp[0].replace(" ","_") + '\n')
#   for sequence in sequences:
#     for element in sequence:
#       for i,item in enumerate(element):
#         if i==0:
#           text_file.write("<" + item + ">" + " ")
#         else:
#           text_file.write(str(item) + " ")
#       text_file.write("-1 ")
#     text_file.write("-2\n")

In [ ]:
# Write the sequences into input file in SPMF format.
with open(r"/content/drive/My Drive/DataMing/SequencesWithTimeNew5.txt", "w") as text_file:
  for sequence in sequences:
    for element in sequence:
      for i,item in enumerate(element):
        if i==0:
          text_file.write("<" + item + ">" + " ")
        else:
          text_file.write(str(item) + " ")
      text_file.write("-1 ")
    text_file.write("-2\n")

**The input file format :** 

It is a text file where each line represents a time-extended sequence from a sequence database. Each line is a list of itemsets, where each itemset has a timestamp represented by a positive integer and each item is represented by a positive integer. Each itemset is first represented by it timestamp between the "<" and "> symbol. Then, the items of the itemset appear separated by single spaces. Finally, the end of an itemset is indicated by "-1". After all the itemsets, the end of a sequence (line) is indicated by the symbol "-2".

### Apply the sequential pattern mining algorithm

**Parameters:**
* minimum support (**minsup**): the minimum number of sequences that should contain a sequential patterns (a positive integer >=0)
* minimum time interval allowed between two succesive itemsets of a sequential pattern (**min_time_interval**) (an integer >=0)
* maximum time interval allowed between two succesive itemsets of a sequential pattern (**max_time_interval**) (an integer >=0)
* minimum time interval allowed between the first itemset and the last itemset of a sequential pattern (**min_whole_interval**) (an integer >=0)
* maximum time interval allowed between the first itemset and the last itemset of a sequential pattern (**max_whole_interval**) (an integer >=0)

**The Fournier-Viger et al., 2008 algorithm** is a sequential pattern mining algorithm combining features from several other sequential pattern mining algorithms. It also offers some original features. In this example, we show how it can be used to discover closed sequential patterns with time-constraints.

Closed sequential patterns is a compact representation of all sequential patterns. Mining closed sequential patterns is important because it can greatly reduce the number of patterns found without loss of information. Using time-constraint is important because it allows to filter unininteresting patterns according to time-related constraints.

Mining closed patterns or using time constraints is also important because it can greatly improve the speed and memory usage when these constraints are used.

In [ ]:
# Fournier08-Closed+time algorithm
spmf = Spmf("Fournier08-Closed+time", spmf_bin_location_dir = r"/content/drive/My Drive/DataMing/", input_filename= r"/content/drive/My Drive/DataMing/SequencesWithTimeNew5.txt",
            output_filename= r"/content/drive/My Drive/DataMing/output.txt", arguments=[0.005,1,30,0,90])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("/content/drive/My Drive/DataMing/outputFournier08_5C.txt")

>/content/drive/My Drive/DataMing/spmf.jar
=============  Algorithm - STATISTICS =============
 Total time ~ 191347 ms
 Frequent sequences count : 21062


                 pattern  sup
0      [<0> 22635 22634]   45
1      [<0> 22635 22138]   25
2            [<0> 22635]  114
3            [<0> 22636]  123
4            [<0> 22637]  172
...                  ...  ...
21057        [<0> 84754]   33
21058        [<0> 72799]   44
21059        [<0> 84789]   32
21060  [<0> 71477 85123]   26
21061        [<0> 71477]  140

[21062 rows x 2 columns]


**The Hirate-Yamana, 2006 algorithm** is an algorithm for discovering frequent sequential patterns respecting some time-constraints to filter uninteresting patterns.

The idea of using time constraints is interesting because it can greatly reduce the number of patterns found and it is also faster and use less memory than if all patterns are discovered.

In [ ]:
# HirateYamana algorithm
spmf = Spmf("HirateYamana", spmf_bin_location_dir = r"/content/drive/My Drive/DataMing/", input_filename= r"/content/drive/My Drive/DataMing/SequencesWithTimeNew5.txt",
            output_filename= r"/content/drive/My Drive/DataMing/output.txt", arguments=[0.005,1,30,0,90])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("/content/drive/My Drive/DataMing/outputHirateYamana_5C.txt")

>/content/drive/My Drive/DataMing/spmf.jar
=============  Algorithm - STATISTICS =============
 Total time ~ 60775 ms
 Frequent sequences count : 21062


                 pattern  sup
0      [<0> 22635 22634]   45
1      [<0> 22635 22138]   25
2            [<0> 22635]  114
3            [<0> 22636]  123
4            [<0> 22637]  172
...                  ...  ...
21057        [<0> 84754]   33
21058        [<0> 72799]   44
21059        [<0> 84789]   32
21060  [<0> 71477 85123]   26
21061        [<0> 71477]  140

[21062 rows x 2 columns]


**The output file format** : 

Each line is a time-extended frequent closed sequential pattern. Each line starts by listing the itemsets of the sequential pattern, where each itemset has a relative timestamp represented by a positive integer between the "<" and "> symbol. Then the timestamp is followed by each item in the itemset, each represented by a positive integer. The items of the itemset appear separated by single spaces and the symbol "-1" indicates the end of an itemset. Finally, after all the itemsets of a sequential pattern, the keyword "#SUP:" is followed by an integer indicating the support of the pattern as a number of sequences.

## Categorize the 'ProdID' with their first three bits.

Most of the similar ProdDescrs can be distinguished by the last two bits of ProdID, so we use the others that is the first three bits to represent the category, but for the feasibility and the giving a name to the category need the domin knowlege for futher study. 

### Model the customer as a sequence of baskets  

In [ ]:
from datetime import datetime
import zipfile

zf = zipfile.ZipFile("/content/drive/My Drive/DataMing/customer_supermarket.csv.zip") 
df = pd.read_csv(zf.open('customer_supermarket.csv'), sep='\t', index_col=0, decimal=',', parse_dates=['BasketDate'])
# Delete all records with 'ProdID' which are not start with a number
df = df[df['ProdID'].str.startswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9'))]
# Delete all return records
df = df[df['BasketID'].str.startswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9'))]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460875 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   BasketID         460875 non-null  object        
 1   BasketDate       460875 non-null  datetime64[ns]
 2   Sale             460875 non-null  float64       
 3   CustomerID       396370 non-null  object        
 4   CustomerCountry  460875 non-null  object        
 5   ProdID           460875 non-null  object        
 6   ProdDescr        460128 non-null  object        
 7   Qta              460875 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 31.6+ MB


In [ ]:
# Take the first five bits of 'ProdID'.
df['ProdID'] = df['ProdID'].apply(lambda x:x[:3])
# Number of total 'ProdID' categories.
len(df['ProdID'].unique()),len(df['ProdDescr'].unique())

(111, 4058)

In [ ]:
# Transform 'BasketDate' into 'Date'.
date_split=df['BasketDate'].astype('str').str.split(' ')
df['Date']=date_split.str[0]
# Parse the variable 'Date' from type 'str' to type 'datetime64[ns]'.
df.loc[:,'Date']=pd.to_datetime(df.loc[:,'Date'], format='%Y-%m-%d', errors='coerce')
Min_date = df.Date.min()
df['Timestamp'] = pd.Series(df['Date'] - Min_date).dt.days
df.tail()

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta,Date,Timestamp
541904,581587,2011-09-12 12:50:00,0.85,12680.0,France,226,PACK OF 20 SPACEBOY NAPKINS,12,2011-09-12,608
541905,581587,2011-09-12 12:50:00,2.10,12680.0,France,228,CHILDREN'S APRON DOLLY GIRL,6,2011-09-12,608
541906,581587,2011-09-12 12:50:00,4.15,12680.0,France,232,CHILDRENS CUTLERY DOLLY GIRL,4,2011-09-12,608
541907,581587,2011-09-12 12:50:00,4.15,12680.0,France,232,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-09-12,608
541908,581587,2011-09-12 12:50:00,4.95,12680.0,France,221,BAKING SET 9 PIECE RETROSPOT,3,2011-09-12,608


In [ ]:
# Generate itemsets.
transactions = df.groupby(['CustomerID','Timestamp','BasketID'])['ProdID'].apply(list).reset_index(name='ProdIDs')
transactions.head()

,CustomerID,Timestamp,BasketID,ProdIDs
0,12346.0,371,541431,[231]
1,12347.0,181,537626,"[851, 223, 714, 224, 227, 227, 227, 227, 227, ..."
2,12347.0,379,542237,"[846, 846, 851, 207, 223, 223, 209, 227, 227, ..."
3,12347.0,392,562032,"[233, 232, 223, 223, 223, 223, 223, 215, 207, ..."
4,12347.0,538,549222,"[223, 223, 223, 223, 206, 230, 217, 225, 231, ..."


In [ ]:
# Generate sequences which sequence lenght large than one
sequences = transactions.groupby(['CustomerID'])['ProdIDs'].apply(list).reset_index()
sequences = sequences[sequences.ProdIDs.str.len()>1]
sequences.to_csv(r'/content/drive/My Drive/DataMing/sequences3.csv', header=True)
sequences = sequences.ProdIDs.to_list()
len(sequences)

2829

In [ ]:
# Write the sequences into input file in SPMF format.
# Giving ProdDescr to corresponding ProdID from text, filter out outliers such as 'nan', 'Damaged', 're-adjustment', 'Unsaleable, destroyed.', 'mailout'. 
with open(r"/content/drive/My Drive/DataMing/SequencesNew3.txt", "w") as text_file:
  for sequence in sequences:
    for element in sequence:
      for i,item in enumerate(element):
        text_file.write(str(item) + " ")
      text_file.write("-1 ")
    text_file.write("-2\n")

### Apply the sequential pattern mining algorithm 

In [ ]:
# PrefixSpan algorithm
spmf = Spmf("PrefixSpan", spmf_bin_location_dir = r"/content/drive/My Drive/DataMing/", input_filename= r"/content/drive/My Drive/DataMing/SequencesNew3.txt",
            output_filename= r"/content/drive/My Drive/DataMing/output.txt", arguments=[0.3])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("/content/drive/My Drive/DataMing/outputPrefixSpan_3.txt")

>/content/drive/My Drive/DataMing/spmf.jar
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 10032 ms
 Frequent sequences count : 1205
 Max memory (mb) : 492.0416717529297
 minsup = 849 sequences.
 Pattern count : 1205


             pattern   sup
0              [848]  1081
1              [849]  1534
2          [849 224]   904
3         [849, 221]   993
4         [849, 224]  1013
...              ...   ...
1200      [233, 233]  1113
1201  [233, 233 233]   898
1202           [234]  1391
1203       [234 233]   937
1204           [235]  1190

[1205 rows x 2 columns]


### Optional (2 points): Extend the algorithm and analysis considering one or more time constraints.

#### Model the customer as a sequence of baskets

In [ ]:
from datetime import datetime
import zipfile

zf = zipfile.ZipFile("/content/drive/My Drive/DataMing/customer_supermarket.csv.zip") 
df = pd.read_csv(zf.open('customer_supermarket.csv'), sep='\t', index_col=0, decimal=',', parse_dates=['BasketDate'])
# Delete all records with 'ProdID' which are not start with a number
df = df[df['ProdID'].str.startswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9'))]
# Delete all return records
df = df[df['BasketID'].str.startswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9'))]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460875 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   BasketID         460875 non-null  object        
 1   BasketDate       460875 non-null  datetime64[ns]
 2   Sale             460875 non-null  float64       
 3   CustomerID       396370 non-null  object        
 4   CustomerCountry  460875 non-null  object        
 5   ProdID           460875 non-null  object        
 6   ProdDescr        460128 non-null  object        
 7   Qta              460875 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 31.6+ MB


In [ ]:
# Categorize the 'ProdID' with their first three bits.
df['ProdID'] = df['ProdID'].apply(lambda x:x[:3])
# Number of total 'ProdID' categories.
len(df['ProdID'].unique())

111

In [ ]:
# Transform 'BasketDate' into 'Date'.
date_split=df['BasketDate'].astype('str').str.split(' ')
df['Date']=date_split.str[0]
# Parse the variable 'Date' from type 'str' to type 'datetime64[ns]'.
df.loc[:,'Date']=pd.to_datetime(df.loc[:,'Date'], format='%Y-%m-%d', errors='coerce')
Min_date = df.Date.min()
df['Timestamp'] = pd.Series(df['Date'] - Min_date).dt.days
df.tail()

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta,Date,Timestamp
541904,581587,2011-09-12 12:50:00,0.85,12680.0,France,226,PACK OF 20 SPACEBOY NAPKINS,12,2011-09-12,608
541905,581587,2011-09-12 12:50:00,2.10,12680.0,France,228,CHILDREN'S APRON DOLLY GIRL,6,2011-09-12,608
541906,581587,2011-09-12 12:50:00,4.15,12680.0,France,232,CHILDRENS CUTLERY DOLLY GIRL,4,2011-09-12,608
541907,581587,2011-09-12 12:50:00,4.15,12680.0,France,232,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-09-12,608
541908,581587,2011-09-12 12:50:00,4.95,12680.0,France,221,BAKING SET 9 PIECE RETROSPOT,3,2011-09-12,608


In [ ]:
# Generate itemsets.
transactions = df.groupby(['CustomerID','Timestamp','BasketID'])['ProdID'].apply(list).reset_index(name='ProdIDs')
transactions.head()

,CustomerID,Timestamp,BasketID,ProdIDs
0,12346.0,371,541431,[231]
1,12347.0,181,537626,"[851, 223, 714, 224, 227, 227, 227, 227, 227, ..."
2,12347.0,379,542237,"[846, 846, 851, 207, 223, 223, 209, 227, 227, ..."
3,12347.0,392,562032,"[233, 232, 223, 223, 223, 223, 223, 215, 207, ..."
4,12347.0,538,549222,"[223, 223, 223, 223, 206, 230, 217, 225, 231, ..."


In [ ]:
# Insert time information for each Basket.
for i in range(len(transactions.ProdIDs)):
 transactions.ProdIDs[i].insert(0, transactions.Timestamp[i].astype(str))
transactions.head()

,CustomerID,Timestamp,BasketID,ProdIDs
0,12346.0,371,541431,"[371, 231]"
1,12347.0,181,537626,"[181, 851, 223, 714, 224, 227, 227, 227, 227, ..."
2,12347.0,379,542237,"[379, 846, 846, 851, 207, 223, 223, 209, 227, ..."
3,12347.0,392,562032,"[392, 233, 232, 223, 223, 223, 223, 223, 215, ..."
4,12347.0,538,549222,"[538, 223, 223, 223, 223, 206, 230, 217, 225, ..."


In [ ]:
# Generate sequences which sequence lenght large than one
sequences = transactions.groupby(['CustomerID'])['ProdIDs'].apply(list).reset_index()
print(len(sequences))
sequences = sequences[sequences.ProdIDs.str.len()>1]
sequences3 = sequences[sequences.ProdIDs.str.len()>2]
sequences4 = sequences[sequences.ProdIDs.str.len()>3]
sequences = sequences.ProdIDs.to_list()
len(sequences),len(sequences3.ProdIDs.to_list()),len(sequences4.ProdIDs.to_list())

4335


(2829, 1998, 1496)

In [ ]:
# Write the sequences into input file in SPMF format.
with open(r"/content/drive/My Drive/DataMing/SequencesWithTimeNew3.txt", "w") as text_file:
  for sequence in sequences:
    for element in sequence:
      for i,item in enumerate(element):
        if i==0:
          text_file.write("<" + item + ">" + " ")
        else:
          text_file.write(str(item) + " ")
      text_file.write("-1 ")
    text_file.write("-2\n")

#### Apply the sequential pattern mining algorithm

In [ ]:
# Fournier08-Closed+time algorithm
spmf = Spmf("Fournier08-Closed+time", spmf_bin_location_dir = r"/content/drive/My Drive/DataMing/", input_filename= r"/content/drive/My Drive/DataMing/SequencesWithTimeNew3.txt",
            output_filename= r"/content/drive/My Drive/DataMing/output.txt", arguments=[0.03,1,30,0,90])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("/content/drive/My Drive/DataMing/outputFournier08_3.txt")

>/content/drive/My Drive/DataMing/spmf.jar
=============  Algorithm - STATISTICS =============
 Total time ~ 595639 ms
 Frequent sequences count : 41596


                   pattern   sup
0                [<0> 350]   108
1            [<0> 230 728]    66
2            [<0> 230 727]    68
3            [<0> 230 714]    94
4      [<0> 230, <28> 224]    58
...                    ...   ...
41591        [<0> 229 475]   530
41592            [<0> 229]  2095
41593            [<0> 900]   134
41594            [<0> 901]   106
41595            [<0> 902]    70

[41596 rows x 2 columns]


In [ ]:
# HirateYamana algorithm
spmf = Spmf("HirateYamana", spmf_bin_location_dir = r"/content/drive/My Drive/DataMing/", input_filename= r"/content/drive/My Drive/DataMing/SequencesWithTimeNew3.txt",
            output_filename= r"/content/drive/My Drive/DataMing/output.txt", arguments=[0.03,1,30,0,90])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("/content/drive/My Drive/DataMing/outputHirateYamana_3.txt")

>/content/drive/My Drive/DataMing/spmf.jar
=============  Algorithm - STATISTICS =============
 Total time ~ 129279 ms
 Frequent sequences count : 41596


                   pattern   sup
0                [<0> 350]   108
1            [<0> 230 728]    66
2            [<0> 230 727]    68
3            [<0> 230 714]    94
4      [<0> 230, <28> 224]    58
...                    ...   ...
41591        [<0> 229 475]   530
41592            [<0> 229]  2095
41593            [<0> 900]   134
41594            [<0> 901]   106
41595            [<0> 902]    70

[41596 rows x 2 columns]


### Discuss the resulting patterns 

In [ ]:
df[df['ProdID'].str.startswith(('721'))].ProdDescr.unique(),len(df[df['ProdID'].str.startswith(('721'))].ProdDescr.unique())

(array(['COLUMBIAN  CUBE CANDLE ', 'COLUMBIAN CANDLE ROUND ',
        'COLUMBIAN CANDLE RECTANGLE', 'COLUMBIAN CANDLE ROUND',
        'COFFEE SCENT PILLAR CANDLE', 'COLUMBIAN CUBE CANDLE',
        'BEST DAD CANDLE LETTERS'], dtype=object), 7)

In [ ]:
df[df['ProdDescr'] == 'Damaged'].ProdID.unique()

array(['374', '222', '231'], dtype=object)